Extracting all the keypoints from the training data


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization # type: ignore
from tensorflow.keras.utils import to_categorical # type: ignore
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau # type: ignore
import time
from tqdm import tqdm  # Progress bar


# GPU Detection and Configuration

,


In [2]:
# ============================================
# GPU DETECTION AND CONFIGURATION (OPTIMIZED)
# ============================================

print("=" * 60)
print("🔍 GPU DETECTION AND CONFIGURATION (OPTIMIZED)")
print("=" * 60)

# Quick TensorFlow version check
print(f"\n📦 TensorFlow Version: {tf.__version__}")

# List all physical devices
physical_devices = tf.config.list_physical_devices()
print(f"All Physical Devices: {physical_devices}")

# GPU detection
print("\n🔍 Detecting GPU devices...")
gpus = tf.config.list_physical_devices('GPU')
print(f"🎮 GPU Devices Found: {len(gpus)}")

if len(gpus) > 0:
    print("\n✅ GPU IS AVAILABLE!")
    
    # Configure GPU memory growth to avoid allocating all memory at once
    print("\n⚙️  Configuring GPU Memory Growth...")
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"   ✅ Memory growth enabled for {len(gpus)} GPU(s)")
        
        # Set GPU as default device
        tf.config.set_visible_devices(gpus[0], 'GPU')
        print(f"   ✅ Using GPU: {gpus[0]}")
        
        # Verify GPU is being used
        print(f"   ✅ GPU Device Name: {gpus[0].name}")
        
    except RuntimeError as e:
        print(f"   ⚠️  Error configuring GPU: {e}")
    
    # Get GPU details
    print("\n📊 GPU Details:")
    try:
        gpu_details = tf.config.experimental.get_device_details(gpus[0])
        print(f"   GPU Details: {gpu_details}")
        if 'device_name' in gpu_details:
            print(f"   Device Name: {gpu_details['device_name']}")
        if 'compute_capability' in gpu_details:
            print(f"   Compute Capability: {gpu_details['compute_capability']}")
    except Exception as e:
        print(f"   ℹ️  GPU details not available: {e}")
    
    # Enable mixed precision training (optional but recommended)
    print("\n⚡ Enabling Mixed Precision Training...")
    try:
        policy = tf.keras.mixed_precision.Policy('mixed_float16')
        tf.keras.mixed_precision.set_global_policy(policy)
        print(f"   ✅ Mixed precision enabled: {policy.name}")
        print("   ℹ️  Note: Output layer will use float32 for numerical stability")
    except Exception as e:
        print(f"   ⚠️  Mixed precision not available: {e}")
        print("   ℹ️  Continuing with float32 precision")
    
    # Verify GPU is available for computation
    print("\n🧪 GPU Verification Test...")
    print(f"   GPU Built with CUDA: {tf.test.is_built_with_cuda()}")
    if gpus:
        print(f"   ✅ GPU Available: True")
        print(f"   ✅ GPU Device Name: {gpus[0].name}")
        
        # Run a simple computation to verify GPU is actually being used
        try:
            with tf.device('/GPU:0'):
                a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
                b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
                c = tf.matmul(a, b)
                
                # Check which device the operation ran on
                device_str = str(c.device)
                print(f"   Operation executed on: {c.device}")
                if 'GPU' in device_str or 'gpu' in device_str.lower():
                    print("   ✅ SUCCESS: GPU is being used for computations!")
                else:
                    print("   ⚠️  WARNING: Operations are running on CPU, not GPU")
        except Exception as e:
            print(f"   ⚠️  GPU test warning: {e}")
            print("   ℹ️  GPU may still work for training")
    else:
        print(f"   ❌ GPU Available: False")
    
    USE_GPU = True
    DEVICE = '/GPU:0'
    print(f"\n🚀 Training will use: {DEVICE}")
    
else:
    print("\n❌ NO GPU FOUND - Will use CPU")
    print("   ⚠️  Training will be slower on CPU")
    USE_GPU = False
    DEVICE = '/CPU:0'
    
    # Quick CUDA check
    print("\n🔍 Checking CUDA support...")
    try:
        if tf.test.is_built_with_cuda():
            print("   ✅ TensorFlow was built with CUDA support")
            print("   ⚠️  But no GPU device was detected")
            print("   💡 Make sure you have:")
            print("      - NVIDIA GPU with CUDA support")
            print("      - CUDA toolkit installed")
            print("      - cuDNN library installed")
            print("      - TensorFlow-GPU version installed")
        else:
            print("   ❌ TensorFlow was NOT built with CUDA support")
    except:
        print("   ⚠️  Could not check CUDA support")

print("\n" + "=" * 60)
print("✅ GPU Configuration Complete!")
print("=" * 60)


🔍 GPU DETECTION AND CONFIGURATION (OPTIMIZED)

📦 TensorFlow Version: 2.10.0
All Physical Devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

🔍 Detecting GPU devices...
🎮 GPU Devices Found: 1

✅ GPU IS AVAILABLE!

⚙️  Configuring GPU Memory Growth...
   ✅ Memory growth enabled for 1 GPU(s)
   ✅ Using GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
   ✅ GPU Device Name: /physical_device:GPU:0

📊 GPU Details:
   GPU Details: {'device_name': 'NVIDIA GeForce MX150', 'compute_capability': (6, 1)}
   Device Name: NVIDIA GeForce MX150
   Compute Capability: (6, 1)

⚡ Enabling Mixed Precision Training...
Your GPU may run slowly with dtype policy mixed_float16 because it does not have compute capability of at least 7.0. Your GPU:
  NVIDIA GeForce MX150, compute capability 6.1
See https://developer.nvidia.com/cuda-gpus for a list of GPUs and their compute capabilities.
If you will use 

In [3]:
# ============================================
# GPU MEMORY MONITORING & OPTIMIZATION TIPS
# ============================================
print("=" * 60)
print("💡 GPU MEMORY MANAGEMENT TIPS")
print("=" * 60)

if USE_GPU:
    print(f"Current batch size: 256 (default for MLP models)")
    print(f"Expected memory usage: ~1.5-2.5 GB (MLP is memory-efficient)")
    
    print("\n💡 MEMORY OPTIMIZATION TIPS:")
    print("1. Close other GPU-intensive applications during training")
    print("2. Close browser tabs with video/graphics (they use GPU)")
    print("3. Monitor memory with: nvidia-smi -l 1 (in separate terminal)")
    print("4. If you get 'Out of Memory' error:")
    print("   - Reduce batch size to 128 or 64")
    print("   - Or close other applications")
    print("5. MLP models are memory-efficient - batch 256 is typically safe")
    
    print("\n📊 To check GPU memory during training:")
    print("   Open Command Prompt/PowerShell and run: nvidia-smi -l 1")
    print("   You should see GPU-Util: 50-100% and Memory-Usage increasing")
else:
    print("⚠️  No GPU detected - memory tips not applicable")
    print("   Training will use CPU memory instead")

print("\n✅ Ready to train with optimized settings!")
print("=" * 60)


💡 GPU MEMORY MANAGEMENT TIPS
Current batch size: 256 (default for MLP models)
Expected memory usage: ~1.5-2.5 GB (MLP is memory-efficient)

💡 MEMORY OPTIMIZATION TIPS:
1. Close other GPU-intensive applications during training
2. Close browser tabs with video/graphics (they use GPU)
3. Monitor memory with: nvidia-smi -l 1 (in separate terminal)
4. If you get 'Out of Memory' error:
   - Reduce batch size to 128 or 64
   - Or close other applications
5. MLP models are memory-efficient - batch 256 is typically safe

📊 To check GPU memory during training:
   Open Command Prompt/PowerShell and run: nvidia-smi -l 1
   You should see GPU-Util: 50-100% and Memory-Usage increasing

✅ Ready to train with optimized settings!


In [4]:
# ============================================
# OPTIMIZED MEDIAPIPE KEYPOINT EXTRACTION
# ============================================

# Check if CSV already exists (skip processing if it does)
CSV_PATH = "asl_mediapipe_keypoints_dataset.csv"
if os.path.exists(CSV_PATH):
    print("=" * 60)
    print("📁 Dataset CSV already exists!")
    print(f"   File: {CSV_PATH}")
    df_existing = pd.read_csv(CSV_PATH)
    print(f"   Samples: {len(df_existing)}")
    print("   ✅ Skipping extraction. Use existing dataset.")
    print("=" * 60)
    print("\n💡 To re-extract, delete the CSV file first.")
else:
    print("=" * 60)
    print("🔍 EXTRACTING MEDIAPIPE KEYPOINTS FROM DATASET")
    print("=" * 60)
    print("⏱️  This will take time depending on dataset size...")
    print("   (Typical ASL dataset: ~29,000 images = 30-60 minutes)")
    print("=" * 60)
    
    # Initialize MediaPipe Hands
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.7)
    
    # Dataset directory
    DATASET_DIR = r'M:\Term 9\Grad\Gradution Current Project - Copy\Sign-Language-Recognition-System-main\Sign-Language-Recognition-System-main\Sign_to_Sentence Project\Asl_Sign_Data\asl_alphabet_train'
    
    # Initialize lists to store extracted data
    landmark_data = []
    labels = []
    
    # Get all image files first (for progress tracking)
    print("\n📂 Scanning dataset...")
    all_images = []
    class_labels = sorted([d for d in os.listdir(DATASET_DIR) if os.path.isdir(os.path.join(DATASET_DIR, d))])
    
    for label in class_labels:
        folder_path = os.path.join(DATASET_DIR, label)
        files = [f for f in os.listdir(folder_path) if f.endswith((".png", ".jpg", ".jpeg"))]
        for file in files:
            all_images.append((label, os.path.join(folder_path, file)))
    
    total_images = len(all_images)
    print(f"   Found {total_images} images across {len(class_labels)} classes")
    print(f"   Classes: {', '.join(class_labels[:10])}{'...' if len(class_labels) > 10 else ''}")
    
    # Process images with progress bar
    print("\n🔄 Processing images...")
    start_time = time.time()
    processed_count = 0
    skipped_count = 0
    
    # Process with progress bar
    for label, img_path in tqdm(all_images, desc="Extracting keypoints", unit="img"):
        try:
            image = cv2.imread(img_path)
            
            # Check if image is valid
            if image is None:
                skipped_count += 1
                continue
            
            # Convert image to RGB (MediaPipe requires RGB)
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Process image with MediaPipe
            results = hands.process(image_rgb)
            
            # If a hand is detected, extract landmarks
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    # Extract landmark points (x, y, z) for 21 keypoints
                    landmarks = np.array([[lm.x, lm.y, lm.z] for lm in hand_landmarks.landmark]).flatten()
                    
                    # Save data
                    landmark_data.append(landmarks)
                    labels.append(label)
                    processed_count += 1
            else:
                skipped_count += 1
                
        except Exception as e:
            skipped_count += 1
            continue
    
    processing_time = time.time() - start_time
    
    # Convert to DataFrame and Save
    print("\n💾 Saving dataset...")
    if len(landmark_data) == 0:
        print("❌ ERROR: No hand landmarks were saved. Check dataset format.")
        df = pd.DataFrame()
    else:
        df = pd.DataFrame(landmark_data)
        df["label"] = labels
        df.to_csv(CSV_PATH, index=False)
        
        print("=" * 60)
        print("✅ EXTRACTION COMPLETE!")
        print("=" * 60)
        print(f"📊 Statistics:")
        print(f"   Total images processed: {total_images}")
        print(f"   Successfully extracted: {processed_count}")
        print(f"   Skipped (no hand detected): {skipped_count}")
        print(f"   Processing time: {processing_time/60:.2f} minutes ({processing_time:.2f} seconds)")
        print(f"   Average time per image: {processing_time/total_images:.3f} seconds")
        print(f"   Dataset saved: {CSV_PATH}")
        print(f"   Dataset size: {len(df)} samples")
        print("=" * 60)

# Load the dataset (either existing or newly created)
if os.path.exists(CSV_PATH):
    df = pd.read_csv(CSV_PATH)
    print(f"\n📦 Dataset loaded: {len(df)} samples")
else:
    print("\n❌ No dataset found. Please run the extraction cell first.")


📁 Dataset CSV already exists!
   File: asl_mediapipe_keypoints_dataset.csv
   Samples: 59801
   ✅ Skipping extraction. Use existing dataset.

💡 To re-extract, delete the CSV file first.

📦 Dataset loaded: 59801 samples


Preprocessing the Mediapipe Keypoints file data


In [ ]:
# Load dataset
df = pd.read_csv("asl_mediapipe_keypoints_dataset.csv")

# Separate features and labels (convert to float32 early to save memory)
X = df.iloc[:, :-1].astype("float32").values
y = df["label"].values

# Encode labels as numbers
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)
num_classes = len(encoder.classes_)

# Split dataset into train/test/validation using encoded labels for stratification
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X,
    y_encoded,
    test_size=0.2,
    random_state=42,
    stratify=y_encoded
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_full,
    y_train_full,
    test_size=0.2,
    random_state=42,
    stratify=y_train_full
)

# Convert labels to one-hot after splitting
X_train = X_train.astype("float32")
X_val = X_val.astype("float32")
X_test = X_test.astype("float32")

y_train = to_categorical(y_train, num_classes=num_classes)
y_val = to_categorical(y_val, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)

print(f"Training samples: {X_train.shape[0]}")
print(f"Validation samples: {X_val.shape[0]}")
print(f"Test samples: {X_test.shape[0]}")


Training samples: 38272
Validation samples: 9568
Test samples: 11961


In [6]:
# Utility to build performant tf.data pipelines
AUTOTUNE = tf.data.AUTOTUNE

def make_dataset(features, labels, batch_size, training=True):
    ds = tf.data.Dataset.from_tensor_slices((features, labels))
    if training:
        buffer_size = min(len(features), 10000)
        ds = ds.shuffle(buffer_size=buffer_size, reshuffle_each_iteration=True)
    ds = ds.batch(batch_size).prefetch(AUTOTUNE)
    return ds


Creation of a Multi-Level-Perceptron Model


In [7]:
# ============================================
# GPU-OPTIMIZED MODEL CREATION
# ============================================

print("🔨 Building MLP Model for GPU Training...")
print(f"   Input shape: {X_train.shape[1]}")
print(f"   Number of classes: {len(np.unique(y_encoded))}")

num_classes = len(np.unique(y_encoded))

# Clear any previous graph to free GPU memory
tf.keras.backend.clear_session()

# Build model with GPU optimization
with tf.device(DEVICE):
    model = Sequential([
        Dense(
            256,
            activation='relu',
            kernel_initializer='he_normal',
            kernel_regularizer=tf.keras.regularizers.l2(1e-4),
            input_shape=(X_train.shape[1],)
        ),
        BatchNormalization(),
        Dropout(0.3),
        Dense(
            128,
            activation='relu',
            kernel_initializer='he_normal',
            kernel_regularizer=tf.keras.regularizers.l2(1e-4)
        ),
        BatchNormalization(),
        Dropout(0.25),
        Dense(
            64,
            activation='relu',
            kernel_initializer='he_normal'
        ),
        Dropout(0.2),
        Dense(num_classes, activation='softmax', dtype='float32')  # Output layer in float32 for stability
    ])
    
    # Use mixed precision friendly optimizer (legacy Adam plays nicer with float16)
    optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)
    
    # Compile with GPU-optimized settings
    model.compile(
        optimizer=optimizer, 
        loss='categorical_crossentropy', 
        metrics=['accuracy']
    )

# Display model summary
print("\n📊 Model Summary:")
model.summary()

# Check if model will use GPU
print(f"\n🎯 Model will train on: {DEVICE}")
if USE_GPU:
    print("   ✅ GPU acceleration enabled")
    print("   ⚡ Mixed precision training: Enabled (if supported)")
else:
    print("   ⚠️  Training on CPU (slower)")


🔨 Building MLP Model for GPU Training...
   Input shape: 63
   Number of classes: 28

📊 Model Summary:
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               16384     
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 batch_normalization_1 (Batc  (None, 128)              512       
 hNormalization)                                                 
                   

Training the MLP Model


In [ ]:
# ============================================
# GPU-OPTIMIZED TRAINING
# ============================================

print("🚀 Starting GPU-Optimized Training...")
print(f"   Training samples: {len(X_train)}")
print(f"   Validation samples: {len(X_val)}")
print(f"   Device: {DEVICE}")

# Report which device will actually be used
if USE_GPU and tf.config.list_physical_devices('GPU'):
    active_gpu = tf.config.list_physical_devices('GPU')[0]
    print(f"   ✓ Training on GPU: {active_gpu.name}")
else:
    print("   ⚠ WARNING: No GPU detected, training will fall back to CPU")

# Optimize batch size based on GPU availability and model complexity
if USE_GPU:
    BATCH_SIZE = 256  # Keeps GPU busy without exhausting 4GB memory
    print(f"   Batch size: {BATCH_SIZE} (optimized for GPU)")
    print("   Expected memory usage: ~1.5-2.5 GB")
else:
    BATCH_SIZE = 64  # Safer batch size for CPU training
    print(f"   Batch size: {BATCH_SIZE} (CPU mode)")
    print("   Tip: Increase to 128 if you have ample CPU RAM")

callbacks = [
    ModelCheckpoint(
        'asl_mediapipe_mlp_model_best.h5',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-7,
        verbose=1
    )
]

# Build efficient tf.data pipelines (keeps GPU fed without CPU bottlenecks)
train_ds = make_dataset(X_train, y_train, BATCH_SIZE, training=True)
val_ds = make_dataset(X_val, y_val, BATCH_SIZE, training=False)

optimizer_name = model.optimizer.__class__.__name__
if hasattr(model.optimizer.learning_rate, 'numpy'):
    lr_value = float(model.optimizer.learning_rate.numpy())
else:
    lr_value = float(model.optimizer.learning_rate)
mixed_precision_status = "Enabled" if USE_GPU else "N/A"

print("\n📊 Training Configuration:")
print(f"  - Optimizer: {optimizer_name} (lr={lr_value:.4e})")
print(f"  - Batch size: {BATCH_SIZE}")
print("  - Callbacks: ModelCheckpoint, EarlyStopping, ReduceLROnPlateau")
print(f"  - Mixed precision: {mixed_precision_status}")
print("  - Validation data: dedicated holdout set (tf.data)")

# Train model with GPU
print("\n⏱️  Training started...")
start_time = time.time()

with tf.device(DEVICE):
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=20,  # Increased epochs, early stopping will prevent overfitting
        callbacks=callbacks,
        verbose=1
    )

training_time = time.time() - start_time
print(f"\n⏱️  Training completed in {training_time:.2f} seconds ({training_time/60:.2f} minutes)")

# Save final model
model.save("asl_mediapipe_mlp_model.h5")
print("✅ Model saved as 'asl_mediapipe_mlp_model.h5'")
print("✅ Best model saved as 'asl_mediapipe_mlp_model_best.h5'")

# Display training summary
if hasattr(history, 'history'):
    final_acc = history.history['accuracy'][-1]
    final_val_acc = history.history['val_accuracy'][-1]
    print(f"\n📊 Final Training Accuracy: {final_acc*100:.2f}%")
    print(f"📊 Final Validation Accuracy: {final_val_acc*100:.2f}%")


IndentationError: expected an indented block (4055112095.py, line 27)

Test Accuracy of the trained Model


In [ ]:
# ============================================
# GPU-ACCELERATED MODEL EVALUATION
# ============================================

print("📊 Loading model for evaluation...")
model = tf.keras.models.load_model("asl_mediapipe_mlp_model.h5")

print(f"🧪 Evaluating on test data (Device: {DEVICE})...")
print(f"   Test samples: {len(X_test)}")

eval_batch_size = 256 if USE_GPU else 128
test_ds = make_dataset(X_test, y_test, eval_batch_size, training=False)

# Evaluate on test data with GPU
start_time = time.time()
with tf.device(DEVICE):
    loss, accuracy = model.evaluate(test_ds, verbose=1)

eval_time = time.time() - start_time
print(f"\n⏱️  Evaluation completed in {eval_time:.4f} seconds")
print(f"📊 Test Loss: {loss:.4f}")
print(f"📊 Test Accuracy: {accuracy * 100:.2f}%")


Testing the Mediapipe Approach for Sign Recognition


In [ ]:
# ============================================
# REAL-TIME INFERENCE (WEBCAM)
# ============================================
# Commit-once-then-wait strategy (prevents letter repetition)
# Control labels match CSV: 'space', 'del' (lowercase, no 'nothing' in ASL dataset)

from collections import deque
import time

print(f"📦 Loading model for inference (Device: {DEVICE})...")
mlp_model = tf.keras.models.load_model("asl_mediapipe_mlp_model.h5")
if USE_GPU:
    print("   ✅ GPU acceleration enabled for inference")

# Load dataset to rebuild LabelEncoder
df = pd.read_csv("asl_mediapipe_keypoints_dataset.csv")
encoder = LabelEncoder()
encoder.fit(df["label"])
print(f"   Encoder classes ({len(encoder.classes_)}): {list(encoder.classes_[:5])}...")

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

# Stabilization settings
STABILIZATION_WINDOW_SIZE = 10
STABILIZATION_THRESHOLD = 7
MIN_CONFIDENCE = 0.70
HOLD_TIME_REQUIRED = 0.8
DISPLAY_WIDTH = 1280
DISPLAY_HEIGHT = 720

# Open webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("❌ Cannot access camera")
else:
    print("✅ Camera opened. Press 'q' to quit, 'c' to clear")
    
    window_name = "Sign Language Recognition (MediaPipe MLP)"
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(window_name, DISPLAY_WIDTH, DISPLAY_HEIGHT)
    
    # State variables
    predicted_sentence = ""
    stabilization_buffer = deque(maxlen=STABILIZATION_WINDOW_SIZE)
    
    # Commit-once-then-wait state
    committed_label = None
    current_sign_label = None
    current_sign_start = None
    waiting_for_change = False
    
    try:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
    
            # Process UNFLIPPED frame with MediaPipe (matches training data)
            frame = cv2.resize(frame, (DISPLAY_WIDTH, DISPLAY_HEIGHT))
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            rgb_frame.flags.writeable = False
            results = hands.process(rgb_frame)
            rgb_frame.flags.writeable = True
    
            display_status = ""
            status_color = (200, 200, 200)
    
            if results.multi_hand_landmarks:
                for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
                    mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
    
                    # Extract landmarks — NO mirroring (matches training data)
                    landmarks = np.array([[lm.x, lm.y, lm.z] for lm in hand_landmarks.landmark])
                    input_data = landmarks.flatten().reshape(1, -1)
                    input_tensor = tf.cast(input_data, tf.float32)
    
                    with tf.device(DEVICE):
                        prediction = mlp_model.predict(input_tensor, verbose=0)
                    predicted_class = np.argmax(prediction)
                    confidence = float(np.max(prediction))
                    predicted_label = encoder.inverse_transform([predicted_class])[0]
    
                    # Skip low confidence
                    if confidence < MIN_CONFIDENCE:
                        display_status = f"{predicted_label} ({confidence:.0%}) Low conf"
                        status_color = (0, 100, 255)
                        break
    
                    # Stability buffer
                    stabilization_buffer.append(predicted_label)
                    buffer_count = stabilization_buffer.count(predicted_label)
                    is_stable = (buffer_count >= STABILIZATION_THRESHOLD and
                                 len(stabilization_buffer) == STABILIZATION_WINDOW_SIZE)
    
                    if not is_stable:
                        progress = buffer_count / STABILIZATION_THRESHOLD * 100
                        display_status = f"{predicted_label} ({confidence:.0%}) Stabilizing {progress:.0f}%"
                        status_color = (0, 255, 255)
                        break
    
                    now = time.time()
    
                    # Check if waiting after a commit
                    if waiting_for_change:
                        if predicted_label == committed_label:
                            display_status = f"{predicted_label} ({confidence:.0%}) ✓ Committed - change sign"
                            status_color = (255, 200, 0)
                            break
                        else:
                            waiting_for_change = False
                            committed_label = None
                            current_sign_label = predicted_label
                            current_sign_start = now
    
                    # Track hold time
                    if predicted_label != current_sign_label:
                        current_sign_label = predicted_label
                        current_sign_start = now
    
                    hold_duration = now - current_sign_start if current_sign_start else 0
    
                    if hold_duration < HOLD_TIME_REQUIRED:
                        hold_pct = hold_duration / HOLD_TIME_REQUIRED * 100
                        display_status = f"{predicted_label} ({confidence:.0%}) Hold: {hold_pct:.0f}%"
                        status_color = (0, 255, 255)
                        break
    
                    # COMMIT — control labels match CSV: 'space', 'del' (lowercase)
                    if predicted_label == "space":
                        if not predicted_sentence.endswith(" "):
                            predicted_sentence += " "
                    elif predicted_label == "del":
                        if predicted_sentence:
                            predicted_sentence = predicted_sentence[:-1]
                    elif predicted_label not in ("nothing",):
                        predicted_sentence += predicted_label
    
                    committed_label = predicted_label
                    waiting_for_change = True
                    current_sign_label = None
                    current_sign_start = None
                    stabilization_buffer.clear()
    
                    display_status = f"{predicted_label} ({confidence:.0%}) ✓ COMMITTED!"
                    status_color = (0, 255, 0)
            else:
                # No hand → full reset
                committed_label = None
                waiting_for_change = False
                current_sign_label = None
                current_sign_start = None
                stabilization_buffer.clear()
                display_status = "No hand detected"
                status_color = (150, 150, 150)
    
            # Flip for selfie-view display
            frame = cv2.flip(frame, 1)
    
            # Status text
            cv2.rectangle(frame, (0, 0), (DISPLAY_WIDTH, 50), (30, 30, 30), -1)
            cv2.putText(frame, display_status, (10, 35),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.9, status_color, 2)
    
            # Bottom bar for sentence
            bar_height = 60
            frame_height, frame_width, _ = frame.shape
            cv2.rectangle(frame, (0, frame_height - bar_height),
                         (frame_width, frame_height), (0, 0, 0), -1)
            cv2.putText(frame, predicted_sentence[-50:], (50, frame_height - 20),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    
            cv2.imshow(window_name, frame)
    
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
            elif key == ord('c'):
                predicted_sentence = ""
                committed_label = None
                waiting_for_change = False
                stabilization_buffer.clear()
                print("🗑️ Sentence cleared")
    
    except KeyboardInterrupt:
        print("\n⚠️ Interrupted by user")
    finally:
        cap.release()
        cv2.destroyAllWindows()
        print(f"\n📝 Final sentence: {predicted_sentence}")
